In [1]:
# 添加搜索路径
import sys
sys.path.insert(0, "/root/mounted/projects/qpair/")

In [12]:
# 加载模块
import os
import gc
import pandas as pd
import pickle
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from matplotlib import pyplot as plt
%matplotlib inline

In [9]:
# 加载全局配置
from config import *

In [4]:
with open(QUESTION_PKL, 'rb') as f:
    qs = pickle.load(f)

qs

,qid,words,chars
0,Q000000,W05733 W05284 W09158 W14968 W07863,L1128 L1861 L2218 L1796 L1055 L0847 L2927
1,Q000001,W17378 W17534 W03249 W01490 W18802,L2214 L1980 L0156 L1554 L2218 L1861 L3019 L010...
2,Q000002,W17378 W08158 W20171 W11246 W14759,L2214 L2350 L2568 L1969 L2168 L0694 L3012 L256...
3,Q000003,W11385 W14103 W02556 W13157 W09749,L0762 L2172 L1971 L2034 L2039 L2271 L1346 L223...
4,Q000004,W17508 W18238 W02952 W18103,L0018 L2321 L1346 L3019 L0104 L0902 L0354 L0362
5,Q000005,W17378 W12908 W19355 W08410 W11922 W14496 W20487,L2214 L1331 L0016 L1132 L2237 L1900 L0578 L195...
6,Q000006,W04346 W17378 W06112 W05733 W18238 W05284 W118...,L2323 L1526 L2214 L0358 L1849 L1128 L1861 L301...
7,Q000007,W16696 W02556 W16564 W03914 W18103,L0366 L2028 L2034 L2039 L0605 L1824 L2292 L015...
8,Q000008,W17378 W11385 W18238 W13157 W12244 W16319 W18448,L2214 L0762 L2172 L3019 L0104 L2271 L1346 L014...
9,Q000009,W14113 W16319 W18399 W03472 W11399 W07863 W18103,L0947 L2323 L2582 L0143 L2600 L2272 L1872 L123...


In [5]:
# 获取最长词序列长度和最长字序列长度
max_wseq_len = max(map(lambda x: len(x.split()) ,qs['words']))
max_cseq_len = max(map(lambda x: len(x.split()) ,qs['chars']))
max_wseq_len, max_cseq_len

(39, 58)

In [6]:
MAX_WSEQ_LEN = 39
MAX_CSEQ_LEN = 58

In [10]:
with open(KN_TRAIN_WIDS_CIDS_PKL, 'rb') as f:
    kn_train_wids_cids = pickle.load(f)
    
kn_train_wids_cids.tail()

,qid1,qid2,words1,chars1,words2,chars2,label
4049435,Q678210,Q210128,"[2739, 7019, 11955, 20577]","[3025, 1890, 162, 1455, 342, 915, 688]","[6855, 17282, 11955, 20577, 2739, 3217, 16541]","[1663, 1890, 481, 1455, 342, 915, 688, 3025, 1...",0
4049436,Q450962,Q210128,"[3340, 9711, 19213, 15579, 10229, 11955, 8781]","[225, 613, 2004, 2534, 225, 2820, 465, 1748, 1...","[6855, 17282, 11955, 20577, 2739, 3217, 16541]","[1663, 1890, 481, 1455, 342, 915, 688, 3025, 1...",0
4049437,Q589034,Q210128,"[19213, 19994, 15670, 19213, 10229, 11955, 205...","[225, 613, 2004, 1532, 225, 1748, 1455, 342, 9...","[6855, 17282, 11955, 20577, 2739, 3217, 16541]","[1663, 1890, 481, 1455, 342, 915, 688, 3025, 1...",0
4049438,Q268900,Q210128,"[11955, 20577, 13246, 18025, 6029]","[1455, 342, 915, 688, 726, 2853, 2269, 1121]","[6855, 17282, 11955, 20577, 2739, 3217, 16541]","[1663, 1890, 481, 1455, 342, 915, 688, 3025, 1...",0
4049439,Q401587,Q210128,"[1818, 18394, 7019, 3337, 20577]","[1262, 1984, 2701, 162, 1737, 2756, 915, 688]","[6855, 17282, 11955, 20577, 2739, 3217, 16541]","[1663, 1890, 481, 1455, 342, 915, 688, 3025, 1...",0


In [11]:
with open(TEST_WIDS_CIDS_PKL, 'rb') as f:
    test_wids_cids = pickle.load(f)

test_wids_cids.tail()

,qid1,qid2,words1,chars1,words2,chars2
24085,Q559873,Q351320,"[14982, 7851, 11759, 2882, 6029, 14284]","[1934, 1670, 2681, 2175, 268, 14, 397, 2427, 1...","[19280, 11095, 11308, 2873, 1553, 4830, 5725, ...","[1112, 1179, 1863, 145, 1863, 2004, 2970, 2976..."
24086,Q469073,Q311144,"[14982, 7851, 2882, 6029, 14284]","[1934, 1670, 2681, 2175, 397, 2427, 1121, 1263]","[9864, 12745, 15083, 10229, 9864, 9520, 18841,...","[2106, 1307, 477, 744, 1748, 2106, 104, 2801, ..."
24087,Q579036,Q516009,"[14982, 2739, 2882]","[1934, 1670, 3025, 1890, 397, 2427]","[2739, 15670, 7445, 6951, 3900]","[3025, 1890, 2004, 1532, 2525, 1273, 2365, 187..."
24088,Q049009,Q652322,"[14982, 9520, 2882, 9416]","[1934, 1670, 104, 397, 2427, 145]","[18025, 13802, 20577, 9520, 2182, 11759]","[2269, 145, 2514, 915, 688, 104, 145, 104, 268..."
24089,Q033965,Q317441,"[9285, 15083]","[137, 477, 744]","[19666, 20569, 3675, 15279, 9864]","[2106, 2269, 1, 1078, 931, 1385, 2106]"


In [17]:
def pad_wseq(item):
    return pad_sequences(item, maxlen=MAX_WSEQ_LEN)

def pad_cseq(item):
    return pad_sequences(item, maxlen=MAX_CSEQ_LEN)

kn_train_wids_cids_words1, kn_train_wids_cids_words2 = kn_train_wids_cids['words1'], kn_train_wids_cids['words2']
kn_train_wids_cids_chars1, kn_train_wids_cids_chars2 = kn_train_wids_cids['chars1'], kn_train_wids_cids['chars2']
test_wids_cids_words1, test_wids_cids_words2 = test_wids_cids['words1'], test_wids_cids['words2']
test_wids_cids_chars1, test_wids_cids_chars2 = test_wids_cids['chars1'], test_wids_cids['chars2']
kn_train_wids_cids_copy = kn_train_wids_cids.copy()
test_wids_cids_copy = test_wids_cids.copy()

In [52]:
kn_train_wids_cids_copy['words1'] = list(pad_sequences(kn_train_wids_cids_words1, maxlen=MAX_WSEQ_LEN)) # kn_train_wids_cids_words1.map(pad_wseq)
kn_train_wids_cids_copy['words2'] = list(pad_sequences(kn_train_wids_cids_words2, maxlen=MAX_WSEQ_LEN)) # kn_train_wids_cids_words2.map(pad_wseq)
kn_train_wids_cids_copy['chars1'] = list(pad_sequences(kn_train_wids_cids_chars1, maxlen=MAX_CSEQ_LEN)) # kn_train_wids_cids_chars1.map(pad_cseq)
kn_train_wids_cids_copy['chars2'] = list(pad_sequences(kn_train_wids_cids_chars2, maxlen=MAX_CSEQ_LEN)) # kn_train_wids_cids_chars2.map(pad_cseq)
kn_train_wids_cids_copy.head()

,qid1,qid2,words1,chars1,words2,chars2,label
0,Q131177,Q112611,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1,Q336221,Q112611,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
2,Q659732,Q112611,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
3,Q686996,Q112611,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
4,Q630751,Q112611,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1


In [53]:
test_wids_cids_copy['words1'] = list(pad_sequences(test_wids_cids_words1, maxlen=MAX_WSEQ_LEN)) # kn_train_wids_cids_words1.map(pad_wseq)
test_wids_cids_copy['words2'] = list(pad_sequences(test_wids_cids_words2, maxlen=MAX_WSEQ_LEN)) # kn_train_wids_cids_words2.map(pad_wseq)
test_wids_cids_copy['chars1'] = list(pad_sequences(test_wids_cids_chars1, maxlen=MAX_CSEQ_LEN)) # kn_train_wids_cids_chars1.map(pad_cseq)
test_wids_cids_copy['chars2'] = list(pad_sequences(test_wids_cids_chars2, maxlen=MAX_CSEQ_LEN)) # kn_train_wids_cids_chars2.map(pad_cseq)
test_wids_cids_copy.head()

,qid1,qid2,words1,chars1,words2,chars2
0,Q345287,Q329078,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Q227821,Q460114,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Q449509,Q525440,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Q088371,Q364132,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Q163532,Q114092,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [54]:
KN_TRAIN_WIDS_CIDS_PADDED_PKL = os.path.join(INTERMEDIATE_DATA_PATH, 'kn_train_wids_cids_padded.pkl')
TEST_WIDS_CIDS_PADDED_PKL = os.path.join(INTERMEDIATE_DATA_PATH, 'test_wids_cids_padded.pkl')

In [55]:
with open(KN_TRAIN_WIDS_CIDS_PADDED_PKL, 'wb') as f:
    pickle.dump(kn_train_wids_cids_copy, f)

In [ ]:
with open(TEST_WIDS_CIDS_PADDED_PKL, 'wb') as f:
    pickle.dump(test_wids_cids_copy, f)